In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from lib.preprocessing.load import read_movies_entrees
from sklearn.preprocessing import MultiLabelBinarizer
from lib.utils.io import read_from_json
import holidays
from vacances_scolaires_france import SchoolHolidayDates

In [4]:
lang_to_keep = ['en', 'fr', 'es', 'it', 'ja', 'de']
country_to_keep = ['FR', 'US', 'GB', 'DE', 'BE', 'CA']
dict_genres = {
    'Drame': 'Drame',
    'Comédie': 'Comédie',
    'Romance': 'Romance',
    'Action': 'Action',
    'Thriller': 'Action',
    'Aventure': 'Action',
    'Crime': 'Action',
    'Guerre': 'Action',
    'Western': 'Action',
    'Familial': 'Familial',
    'Animation': 'Familial',
    'Fantastique': 'Fantastique',
    'Science-Fiction': 'Fantastique',
    'Horreur': 'Horreur',
    'Mystère': 'Other',
    'Musique': 'Other',
    'Histoire': 'Other',
    'Documentaire': 'Other',
    'Téléfilm': 'Other'
}

In [5]:
def read_movies_features(path: str) -> pd.DataFrame:
    '''
    Read the movie features dataset 
    and casts it as an usable pandas DataFrame
    N.B: Fields that are not yet used are commented
    Parameters
    ----------
    path: str
        path to the dataset
    Returns
    -------
    df: pd.DataFrame
        Data as DataFrame
    '''
    features = read_from_json(path)
    features = [
        {
            "is_adult": item['adult'],
            "is_part_of_collection": not not item['belongs_to_collection'],
            "collection_name": item['belongs_to_collection']['name'] if item['belongs_to_collection'] != {} else None, # Currently simple bool, may be interesting to use a more complex feature later
            "budget": item['budget'],
            "genres": [ genre['name'] for genre in item['genres'] ], 
            "original_language": item['original_language'],
            "overview": item['overview'], # Not used yet. Blob of text
            "production_countries": [ country['iso_code'] for country in item['production_countries'] ],
            "languages": [ language['iso_code'] for language in item['languages'] ],
            "tagline": item['tagline'], # Not used yet. Blob of text
            "runtime": item['runtime'],
            "cast": item['cast'], # Not used yet. List of dicts with actor gender, name, id...
            "id": int(item['id'])
        } for item in features
    ]
    return pd.DataFrame(features)


def reduce_lang_categories(lang_list, lang_to_keep=lang_to_keep):
    return list(set([el if el in lang_to_keep else 'other' for el in lang_list]))


def reduce_country_categories(country_list, country_to_keep=country_to_keep):
    return list(set([el if el in country_to_keep else 'OTHER' for el in country_list]))


def reduce_genre_categories(genre_list, dict_genres=dict_genres):
    return list(set([dict_genres[el] for el in genre_list]))


def apply_cos(df: pd.DataFrame,
              x: str, col_name: str, period: int) -> pd.DataFrame:
    """ Cos function on a column, for a specified period
    """
    df[col_name] = 2 * np.cos(2 * np.pi * df[x] / period)
    return df

# 1. Load data

In [6]:
# We first read the french movies "entrées"
bo = read_movies_entrees('data/french-box-office-29nov2020.json')
# Then fetch their main features. Note that it seems not all movies were found in the feature database
features = read_movies_features('data/movie-features-29nov2020.json')
# Let's merge both dataframes
data = pd.merge(bo, features, on='id')

# 2. Data cleaning (based on previous EDA)

In [7]:
# Remove rows with missing values for sales
data = data.loc[data['sales'] > 0]
# Replace zero values for budget with median value
data.loc[data['budget'] == 0, 'budget'] = np.median(data.loc[data['budget'] != 0]['budget'])
# Runtime: fill with the mean
runtime_mean = np.mean(data.loc[(data['runtime'] != 0) & (data['runtime'].isnull() == False)]['runtime'])
data.loc[(data['runtime'].isnull() == True) & (data['runtime'] == 0), 'runtime'] = runtime_mean

# Reduce number of categories for original language
data['original_language'] = data['original_language'].map(lambda x: x if x in lang_to_keep else 'other')

# Reduce number of categories for languages
data['languages'] = data['languages'].map(lambda x: reduce_lang_categories(x))

# Reduce number of categories for production countries
data['production_countries'] = data['production_countries'].map(lambda x: reduce_country_categories(x))

# Reduce number of categories for genres
data['genres'] = data['genres'].map(lambda x: reduce_genre_categories(x))
data = data.loc[data.astype(str)['genres'] != '[]']

In [8]:
# Encode is_part_of_collection into numerical
dict_collection = {
    True: 1,
    False: 0
}
data['is_part_of_collection'] = data['is_part_of_collection'].map(dict_collection)

In [9]:
# Fill some missing values for production countries based on original language
data.loc[(data.astype(str)['production_countries'] == '[]') & (
    data['original_language'] == 'fr'), 'production_countries'] = ['FR']
data.loc[(data.astype(str)['production_countries'] == '[]') & (
    data['original_language'] == 'en'), 'production_countries'] = ['US']
data.loc[(data.astype(str)['production_countries'] == '[]'), 'production_countries'] = ['OTHER']

data.loc[(data.astype(str)['languages'] == '[]'), 'languages'] = data.loc[(
    data.astype(str)['languages'] == '[]'), 'original_language'].map(lambda x: [x])

# 3. Feature engineering

### One-hot encoding / Multilabel encoding

In [10]:
# Original language
data_final = pd.get_dummies(data, prefix='original_lang', columns=['original_language'], drop_first=True)
data_final = data_final.set_index('id')

# Languages
mlb = MultiLabelBinarizer()
df_lang = pd.DataFrame(mlb.fit_transform(data_final['languages']), columns=mlb.classes_, index=data_final.index)
df_lang.columns = ['available_lang_' + col for col in df_lang.columns]

# Genres
mlb = MultiLabelBinarizer()
df_genre = pd.DataFrame(mlb.fit_transform(data_final['genres']), columns=mlb.classes_, index=data_final.index)

# Production countries
mlb = MultiLabelBinarizer()
df_country = pd.DataFrame(mlb.fit_transform(data_final['production_countries']), columns=mlb.classes_, index=data_final.index)
df_country.columns = ['prod_' + col for col in df_country.columns]

In [11]:
data_final = pd.merge(data_final, df_lang, left_index=True, right_index=True) \
               .merge(df_genre, left_index=True, right_index=True) \
               .merge(df_country, left_index=True, right_index=True)

### Features holidays

In [12]:
# Load school holidays for France
fr_holidays = SchoolHolidayDates()
df_vacances = pd.DataFrame()
for year in list(set(data_final['year'])):
    df_vacances = pd.concat([df_vacances, pd.DataFrame.from_dict(fr_holidays.holidays_for_year(year)).T])

# Load bank holidays for France
df_jf = pd.DataFrame()
for year in list(set(data_final['year'])):
    df_jf = pd.concat([df_jf, pd.DataFrame([
        {'date': el[0], 'jour_ferie': el[1]} for el in sorted(holidays.FRA(years=year).items())])])

# Merge school and bank holidays
df_holidays = pd.merge(df_vacances, df_jf, how='outer', on='date')

In [13]:
# Encode holidays
df_holidays['jour_ferie'] = df_holidays['jour_ferie'].map(lambda x: 1 if str(x) != 'nan' else 0)

dict_map_vac = {
    True: 1,
    False: 0
}
df_holidays['vacances_zone_a'] = df_holidays['vacances_zone_a'].map(dict_map_vac)
df_holidays['vacances_zone_b'] = df_holidays['vacances_zone_b'].map(dict_map_vac)
df_holidays['vacances_zone_c'] = df_holidays['vacances_zone_c'].map(dict_map_vac)

In [14]:
# Merge with main dataframe
df_holidays['date'] = df_holidays['date'].map(lambda x: str(x))
data_final_cal = pd.merge(data_final, df_holidays, how='left', left_on='release_date', right_on='date').fillna(0)

In [15]:
# Create global holidays feature
data_final_cal['holidays'] = data_final_cal['vacances_zone_a'] + data_final_cal[
    'vacances_zone_b'] + data_final_cal['vacances_zone_c'] + data_final_cal['jour_ferie']

# Drop useless features
col_to_drop = ['nom_vacances', 'date']
data_final_cal = data_final_cal.drop(col_to_drop, axis=1)

### Features calendaires (month, cos_month)

In [16]:
# Create two calendar features for month
data_final_cal['month'] = data_final_cal['release_date'].map(lambda x: int(x[5:7]))
data_final_cal = apply_cos(data_final_cal, 'month', 'cos_month', 12)

### Is part of collection 

In [17]:
# Exclude collections with only one movie
df_count_col = data_final_cal.groupby(['collection_name']).count().reset_index()
not_collection = list(set(df_count_col.loc[df_count_col['year'] < 2]['collection_name']))

data_final_cal.loc[data_final_cal['collection_name'].isin(not_collection), 'is_part_of_collection'] = 0
data_final_cal.loc[data_final_cal['collection_name'].isin(not_collection), 'collection_name'] = None

In [18]:
# Get the number of movies per collection
map_col_count = dict(data_final_cal['collection_name'].value_counts())
del map_col_count[0]
data_final_cal['nb_movie_collection'] = data_final_cal['collection_name'].map(map_col_count)

In [19]:
# Compute rolling sales for previous movies per collection
df_collection = data_final_cal.loc[data_final_cal['is_part_of_collection'] == 1]
df_collection['rolling_sales_collection'] = df_collection.sort_values(by=['collection_name', 'release_date']) \
             .groupby('collection_name')['sales'] \
             .transform(lambda x: x.rolling(10, 1).mean().shift())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
# Merge with main dataframe
cols = ['year', 'title', 'release_date', 'collection_name', 'sales', 'rolling_sales_collection']
df_all = pd.merge(data_final_cal, df_collection[cols], how = 'left', 
                  on = ['year', 'title', 'release_date', 'collection_name', 'sales']).fillna(0)

In [21]:
df_all.head(2)

,year,title,sales,release_date,is_adult,is_part_of_collection,collection_name,budget,genres,overview,...,prod_US,vacances_zone_a,vacances_zone_b,vacances_zone_c,jour_ferie,holidays,month,cos_month,nb_movie_collection,rolling_sales_collection
0,2019,Maléfique : Le Pouvoir du Mal,786485,2019-10-16,False,0,0,185000000.0,"[Fantastique, Action, Familial]",Cinq années après la conjuration de la malédic...,...,1,0.0,0.0,0.0,0.0,0.0,10,1.000000,0.0,0.0
1,2019,Nous finirons ensemble,1261701,2019-05-01,False,1,Les Petits Mouchoirs - Saga,25000000.0,"[Comédie, Drame]","Préoccupé, Max est parti dans sa maison au bor...",...,0,0.0,0.0,1.0,1.0,2.0,5,-1.732051,2.0,1369812.0


### Text features (title, cast, overview, tagline)

# 4. Save file

In [265]:
to_drop = ['genres', 'production_countries', 'languages', 'budget_null', 'is_adult', 'title', 'collection_name',
           'overview', 'tagline', 'cast']
df_all = df_all.drop(to_drop, axis=1).reset_index()

In [268]:
df_all.to_csv('data_prepared_ponts_v2.csv', index=None)